<font color="Teal" face="arial">
  <h2 align="left"><i>Módulo 1. Introducción a la IA en los mercados financieros</i></h2></font>
  <font color="Black" face="arial">
  <h3 align="left"><i>Ejemplo. Algoritmos genéticos para maximizar un portafolio</i></h3>
  <h5 align="left"><i>Dr. César Galindo (UP-UNAM)</i></h5>
</font>



In [8]:
import numpy as np
import random
import yfinance as yf

# ----------------------------
# 1. Descargar datos de activos (ejemplo: México y USA)
# "WALMEX.MX" Walmart
# "BIMBOA.MX" Grupo Bimbo, S.A.B. de C.V
# "TSLA" Tesla, Inc.
# "AAPL" Apple Inc.
# ----------------------------
tickers = ["WALMEX.MX", "BIMBOA.MX", "TSLA", "AAPL"]

data = yf.download(tickers, start="2022-01-01", end="2023-12-31", auto_adjust=False)

# Usamos 'Adj Close' si existe, si no usamos 'Close'
if "Adj Close" in data.columns:
    prices = data["Adj Close"]
else:
    prices = data["Close"]

data


[*********************100%***********************]  4 of 4 completed


Price        Adj Close                                         Close  \
Ticker            AAPL  BIMBOA.MX        TSLA  WALMEX.MX        AAPL   
Date                                                                   
2022-01-03  178.270325  59.000099  399.926666  67.551559  182.009995   
2022-01-04  176.007782  59.415665  383.196655  67.104126  179.699997   
2022-01-05  171.326035  57.602306  362.706665  67.396324  174.919998   
2022-01-06  168.466003  60.152332  354.899994  66.702370  172.000000   
2022-01-07  168.632507  60.898460  342.320007  66.766281  172.169998   
...                ...        ...         ...        ...         ...   
2023-12-22  191.788742  86.011513  252.539993  67.965256  193.600006   
2023-12-26  191.243912  85.604057  256.609985  68.349785  193.050003   
2023-12-27  191.342987  85.060783  261.440002  69.301491  193.149994   
2023-12-28  191.768951  84.721237  253.179993  68.724701  193.580002   
2023-12-29  190.728760  83.295151  248.479996  68.628571  192.529999   

Price                                               High             ...  \
Ticker      BIMBOA.MX        TSLA  WALMEX.MX        AAPL  BIMBOA.MX  ...   
Date                                                                 ...   
2022-01-03  62.470001  399.926666  73.980003  182.880005  63.790001  ...   
2022-01-04  62.910000  383.196655  73.489998  182.940002  63.200001  ...   
2022-01-05  60.990002  362.706665  73.809998  180.169998  62.380001  ...   
2022-01-06  63.689999  354.899994  73.050003  175.300003  63.990002  ...   
2022-01-07  64.480003  342.320007  73.120003  174.139999  64.650002  ...   
...               ...         ...        ...         ...        ...  ...   
2023-12-22  88.660004  252.539993  70.699997  195.410004  88.959999  ...   
2023-12-26  88.239998  256.609985  71.099998  193.889999  89.839996  ...   
2023-12-27  87.680000  261.440002  72.089996  193.500000  89.230003  ...   
2023-12-28  87.330002  253.179993  71.489998  194.660004  89.120003  ...   
2023-12-29  85.860001  248.479996  71.389999  194.399994  88.220001  ...   

Price              Low                   Open                         \
Ticker            TSLA  WALMEX.MX        AAPL  BIMBOA.MX        TSLA   
Date                                                                   
2022-01-03  378.679993  73.800003  177.830002  63.080002  382.583344   
2022-01-04  374.350006  72.830002  182.630005  62.930000  396.516663   
2022-01-05  360.336670  73.110001  179.610001  62.090000  382.216675   
2022-01-06  340.166656  72.239998  172.699997  60.900002  359.000000   
2022-01-07  336.666656  72.230003  172.889999  63.000000  360.123322   
...                ...        ...         ...        ...         ...   
2023-12-22  251.369995  70.209999  195.179993  88.389999  256.760010   
2023-12-26  252.910004  70.699997  193.610001  89.269997  254.490005   
2023-12-27  257.519989  70.309998  192.490005  88.290001  258.350006   
2023-12-28  252.710007  70.940002  194.139999  87.650002  263.660004   
2023-12-29  247.429993  71.220001  193.899994  87.489998  255.100006   

Price                       Volume                                      
Ticker      WALMEX.MX         AAPL  BIMBOA.MX         TSLA   WALMEX.MX  
Date                                                                    
2022-01-03  76.489998  104487900.0   543738.0  103931400.0  10816485.0  
2022-01-04  74.199997   99310400.0  1220353.0  100248300.0  13394249.0  
2022-01-05  73.320000   94537600.0  1492518.0   80119800.0  10971156.0  
2022-01-06  73.610001   96904000.0  2454987.0   90336600.0  10997537.0  
2022-01-07  72.919998   86709100.0  1833800.0   84164700.0  10146926.0  
...               ...          ...        ...          ...         ...  
2023-12-22  70.709999   37149600.0   438438.0   93370100.0   6591886.0  
2023-12-26  70.699997   28919300.0   474925.0   86892400.0   1939249.0  
2023-12-27  71.070000   48087700.0  1371665.0  106494400.0   6887436.0  
2023-12-28  71.709999   34049900.0   9105

In [9]:
# Rendimientos diarios
returns = prices.pct_change().dropna()
returns


/tmp/ipython-input-2308520581.py:2: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  returns = prices.pct_change().dropna()


Ticker,AAPL,BIMBOA.MX,TSLA,WALMEX.MX
Date,,,,
2022-01-04,-0.012692,0.007043,-0.041833,-0.006624
2022-01-05,-0.026600,-0.030520,-0.053471,0.004354
2022-01-06,-0.016693,0.044270,-0.021523,-0.010297
2022-01-07,0.000988,0.012404,-0.035447,0.000958
2022-01-10,0.000116,-0.017835,0.030342,-0.013403
...,...,...,...,...
2023-12-22,-0.005548,0.005671,-0.007701,0.003264
2023-12-26,-0.002841,-0.004737,0.016116,0.005658
2023-12-27,0.000518,-0.006346,0.018822,0.013924


In [10]:
mean_returns = returns.mean() * 252   # rendimiento anualizado
mean_returns

,0
Ticker,
AAPL,0.073801
BIMBOA.MX,0.216933
TSLA,-0.056167
WALMEX.MX,0.037632


In [11]:
cov_matrix = returns.cov() * 252      # matriz de covarianzas
cov_matrix


Ticker,AAPL,BIMBOA.MX,TSLA,WALMEX.MX
Ticker,,,,
AAPL,0.081913,0.014269,0.097911,0.010929
BIMBOA.MX,0.014269,0.097361,0.024667,0.011982
TSLA,0.097911,0.024667,0.350830,0.016975
WALMEX.MX,0.010929,0.011982,0.016975,0.060160


In [12]:
# ----------------------------
# 2. Función de fitness: Ratio de Sharpe
# ----------------------------
rf = 0.05  # tasa libre de riesgo
def fitness(pesos):
    ret = np.dot(pesos, mean_returns)
    vol = np.sqrt(np.dot(pesos.T, np.dot(cov_matrix, pesos)))
    sharpe = (ret - rf) / vol
    return sharpe

In [13]:

# ----------------------------
# 3. Algoritmo Genético
# ----------------------------
def crear_individuo(n_assets):
    w = np.random.rand(n_assets)
    return w / np.sum(w)

def crossover(p1, p2):
    alpha = np.random.rand()
    hijo = alpha * p1 + (1-alpha) * p2
    return hijo / np.sum(hijo)

def mutacion(ind, tasa=0.1):
    if np.random.rand() < tasa:
        i, j = np.random.choice(len(ind), 2, replace=False)
        ind[i], ind[j] = ind[j], ind[i]
    return ind / np.sum(ind)

# Evolución
n_assets = len(tickers)
pop_size = 100
gens = 200

In [14]:
poblacion = [crear_individuo(n_assets) for _ in range(pop_size)]
mejor, mejor_fit = None, -999

for g in range(gens):
    fitness_vals = [fitness(ind) for ind in poblacion]
    orden = np.argsort(fitness_vals)[::-1]
    poblacion = [poblacion[i] for i in orden]
    if fitness_vals[orden[0]] > mejor_fit:
        mejor, mejor_fit = poblacion[0], fitness_vals[orden[0]]
    nueva = poblacion[:10]  # elitismo
    while len(nueva) < pop_size:
        p1, p2 = random.sample(poblacion[:30], 2)
        hijo = crossover(p1, p2)
        hijo = mutacion(hijo)
        nueva.append(hijo)
    poblacion = nueva


# ----------------------------
# 4. Resultados
# ----------------------------
print("Mejores pesos del portafolio:")
for t, w in zip(tickers, mejor):
    print(f"{t}: {w:.2%}")
print(f"Mejor Sharpe ratio: {mejor_fit:.2f}")

Mejores pesos del portafolio:
WALMEX.MX: 26.49%
BIMBOA.MX: 58.00%
TSLA: 7.75%
AAPL: 7.76%
Mejor Sharpe ratio: 0.41
